### 使用 HR 完成螺旋波，耦合方式使用差分法

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from numba import njit, prange
from import_fun import HR, Diffusion2D, show_spiral_wave, FlowVelocity

In [2]:
# 开启交互式画图
mpl.use('TkAgg') # TkAgg Qt5Agg
plt.ion()  # 开启交互模式

In [3]:
Nx = 200
Ny = 200
method = "euler"
dt = 0.01

In [4]:
# 生成节点，初始值设定
nodes = HR(N=Nx*Ny, method=method, dt=dt)
nodes.vars_nodes[0, :] = -1.31742
nodes.vars_nodes[1, :] = -7.67799
nodes.vars_nodes[2, :] = 1.12032

# 生成一个方阵视图，这个视图与原始数组共享内存
x_view = nodes.vars_nodes[0, :].reshape(Nx, Ny)
y_view = nodes.vars_nodes[1, :].reshape(Nx, Ny)
z_view = nodes.vars_nodes[2, :].reshape(Nx, Ny)

# 设定楔形初始值
x_view[91:93, 0:100] = 2.
y_view[91:93, 0:100] = 2.
z_view[91:93, 0:100] = -1.

x_view[94:96, 0:100] = 0.
y_view[94:96, 0:100] = 0.
z_view[94:96, 0:100] = 0.

x_view[97:99, 0:100] = -1.
y_view[97:99, 0:100] = -1.
z_view[97:99, 0:100] = 2.

nodes.params_nodes["Iex"] = 1.315
nodes.params_nodes["s"] = 3.9
nodes.spiking = False    # 关掉峰值探测器

In [5]:
# 设定动态显示器
spiral_wave = show_spiral_wave(nodes.vars_nodes[0], Nx, Ny, save_gif=False)

In [6]:
# 设定扩散耦合
syn = Diffusion2D(D=2., boundary="No_flow", adjacency=4)

In [7]:
for i in range(1000_00):
    spiral_wave(i, nodes.t, show_interval=1000)
    Isyn = syn(x_view)
    nodes(Isyn)

In [8]:
# spiral_wave.save_image()
# spiral_wave.show_final()

In [9]:
# plt.imshow(x_view)
# plt.colorbar()
# plt.show()

In [10]:
fvelocity = FlowVelocity(x_view.copy(), alpha=12.)

for i in range(2000_00):
    if i%10_00 == 5_00:
        fvelocity.V_dalay = x_view.copy()
    if i%10_00 == 1:
        dv_dx, dv_dy = fvelocity(x_view)
    Isyn = syn(x_view)
    nodes(Isyn)

In [11]:
# dv_dx, dv_dy = fvelocity(V_view)
dv_dx_reduced, dv_dy_reduced = fvelocity.reduce_density(dv_dx, dv_dy, v_x_size=40, v_y_size=40)

In [12]:
dv_dx1, dv_dy1 = dv_dx, dv_dy
# dv_dx1, dv_dy1 = dv_dx_reduced, dv_dy_reduced

In [13]:
Ny, Nx = dv_dx1.shape  # 注意形状是 (行, 列) = (y, x)
x = np.arange(Nx)
y = np.arange(Ny)
X, Y = np.meshgrid(x, y)
step = 2
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.quiver(X[::step, ::step], Y[::step, ::step],
               -dv_dy1[::step, ::step], -dv_dx1[::step, ::step], color="b") # , scale=500
ax2.imshow(x_view, cmap='jet', origin="lower", aspect="auto")
# x_min, x_max = 35, 65
# y_min, y_max = 35, 65
# ax1.set_xlim(x_min, x_max)
# ax1.set_ylim(y_min, y_max)
# ax2.set_xlim(x_min, x_max)
# ax2.set_ylim(y_min, y_max)
plt.show()